[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/giswqs/gee-tutorials/blob/master/ImageCollection/reducing_image_collection.ipynb)

**Reducing an ImageCollection**

To composite images in an `ImageCollection`, use `imageCollection.reduce()`. This will composite all the images in the collection to a single image representing, for example, the min, max, mean or standard deviation of the images. (See the Reducers section for more information about reducers). For example, to create a median value image from a collection:

## Create an interactive map

In [1]:
import ee
import geemap

In [2]:
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Compute a median image

In [4]:
# Load a Landsat 8 collection for a single path-row.
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filter(ee.Filter.eq('WRS_PATH', 44)) \
    .filter(ee.Filter.eq('WRS_ROW', 34)) \
    .filterDate('2014-01-01', '2015-01-01')

# Compute a median image and display.
median = collection.median()
Map.setCenter(-122.3578, 37.7726, 12)
Map.addLayer(median, {'bands': ['B4',  'B3',  'B2'], 'max': 0.3}, 'median')

In [5]:
collection.size().getInfo()

19

In [6]:
collection.aggregate_array("system:id").getInfo()

['LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140113',
 'LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140129',
 'LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140214',
 'LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318',
 'LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140403',
 'LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140419',
 'LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140505',
 'LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140521',
 'LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140606',
 'LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140622',
 'LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140708',
 'LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140724',
 'LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140809',
 'LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140825',
 'LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140910',
 'LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140926',
 'LANDSAT/LC08/C01/T1_TOA/LC08_044034_20141012',
 'LANDSAT/LC08/C01/T1_TOA/LC08_044034_20141028',
 'LANDSAT/LC08/C01/T1_TOA/LC08_044034_20141231']

## Use median reducer

At each location in the output image, in each band, the pixel value is the median of all unmasked pixels in the input imagery (the images in the collection). In the previous example, `median()` is a convenience method for the following call:

In [8]:
# Reduce the collection with a median reducer.
median = collection.reduce(ee.Reducer.median())

# Display the median image.
Map.addLayer(median,
             {'bands': ['B4_median',  'B3_median',  'B2_median'], 'max': 0.3},
             'also median')
Map

Map(bottom=102416.0, center=[36.910372213522535, -121.87670918896643], controls=(WidgetControl(options=['posit…

## Create an image composite

In [9]:
states = ee.FeatureCollection('TIGER/2018/States')
Map.addLayer(states, {}, "US States")

In [10]:
ca = states.filter(ee.Filter.eq("NAME", "California"))
Map.addLayer(ca, {}, "California")

In [11]:
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(ca) \
    .filterDate('2020-01-01', '2021-01-01')

In [12]:
collection.size().getInfo()

838

In [13]:
image = collection.median().clip(ca)
Map.addLayer(image, {'bands': ['B4',  'B3',  'B2'], 'max': 0.3}, 'Landsat 2020')